In [ ]:
# This is the first cell, where image_path is defined
from pathlib import Path

# Define the path to the data folder
data_path = Path("data/")
image_path = data_path / "fgvc_aircraft"

In [ ]:
import os
import sys
import zipfile
import requests

# 1️⃣ Mount Google Drive (if using for storage)
use_gdrive = False  # Set to True if dataset is stored in Google Drive
if use_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

# 2️⃣ Clone your GitHub repo if it's not already present
repo_url = "https://github.com/jmand626/PyTorchMLEngine-Custom-Dataset-Project.git"
repo_name = "PyTorchMLEngine-Custom-Dataset-Project"

if not os.path.exists(repo_name):
    print(f"Cloning {repo_url}...")
    !git clone {repo_url}
else:
    print(f"Repository {repo_name} already exists.")

# 3️⃣ Change to repo directory ONLY ONCE
os.chdir(repo_name) # This line sets the working directory

# 4️⃣ Add project files to sys.path so imports work
sys.path.append(os.getcwd())
print("Project directory added to sys.path")

# 5️⃣ Ensure necessary dependencies are installed
try:
    import torchinfo
except ImportError:
    print("Installing torchinfo...")
    !pip install -q torchinfo

# 6️⃣ Download FGVC Aircraft dataset if missing
dataset_url = "https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz"
dataset_tar = data_path / "fgvc-aircraft-2013b.tar.gz"
dataset_folder = data_path / "fgvc-aircraft-2013b"

if dataset_folder.exists():
    print("Dataset already exists.")
else:
    print("Downloading FGVC Aircraft dataset...")
    data_path.mkdir(parents=True, exist_ok=True)
    response = requests.get(dataset_url, stream=True)
    with open(dataset_tar, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Extracting dataset...")
    !tar -xzf {dataset_tar} -C {data_path}
    os.remove(dataset_tar)
    print("Dataset extraction complete.")

Cloning https://github.com/jmand626/PyTorchMLEngine-Custom-Dataset-Project.git...
Cloning into 'PyTorchMLEngine-Custom-Dataset-Project'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 24 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 21.93 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Project directory added to sys.path
Installing torchinfo...
Extracting dataset...
Dataset extraction complete.


In [ ]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

In [ ]:
!ls

computer_vision_test_main.py  data		 model_backbone.py  setup_dataholders.py
create_custom_dataset.py      firsttry_model.py  README.md


In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Now hopefully we can continously use the previous setup code whenever we want to use this dataset again.

In [ ]:
# ipython-input-12-e84a44c78b2d
# Assume the dataset is extracted to 'data/fgvc-aircraft-2013b'
# and images are in 'data/fgvc-aircraft-2013b/data/images'
from pathlib import Path # Make sure to import Path
import os # Import os to get current working directory

# Fix: Update paths to include the subfolder where the dataset was downloaded and extracted
# Use os.path.join to create platform-independent paths
# The issue was train_dir and test_dir were pointing to the wrong location.
# They should point to the parent directory containing the class folders.
train_dir = Path(os.path.join(os.getcwd(), "data/fgvc-aircraft-2013b/data")) # Corrected path
test_dir = Path(os.path.join(os.getcwd(), "data/fgvc-aircraft-2013b/data"))  # Corrected path, assuming test images are in the same location


# Print the resolved paths to verify they are correct
print("Train directory:", train_dir)
print("Test directory:", test_dir)

Train directory: /content/PyTorchMLEngine-Custom-Dataset-Project/data/fgvc-aircraft-2013b/data
Test directory: /content/PyTorchMLEngine-Custom-Dataset-Project/data/fgvc-aircraft-2013b/data


Now we continue on to creating our datasets and dataloaders. An important issue is that we have to ensure that the data that we feed into our pretrained model must be formatted in the same way as the data inputted when training the model (helps performance immeasurably). There is a certain way that all models from torchvision.models require, and we will do that.

It is detailed in this page: https://docs.pytorch.org/vision/0.8/models.html

In [ ]:
import torchvision.transforms as transforms
import importlib
import setup_dataholders
importlib.reload(setup_dataholders)
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])

In [ ]:
import os
from pathlib import Path
import shutil

# Define the path to the images directory
images_dir = Path("data/fgvc-aircraft-2013b/data/images")

# Define the path to the labels file (variants.txt)
labels_file = Path("data/fgvc-aircraft-2013b/data/variants.txt")

# Create a dictionary mapping image names to class labels
image_to_class = {}
with open(labels_file, "r") as f:
    for line in f:
        # Split the line using space as delimiter, but handle cases with missing values
        parts = line.strip().split(" ", 1)

        # Check if the line has enough values to unpack
        if len(parts) == 2:
            image_name, class_label = parts
            image_to_class[image_name + ".jpg"] = class_label
        else:
            # Handle cases with missing values (e.g., print a warning or skip)
            print(f"Warning: Skipping line with missing data: {line.strip()}")

# Create class subfolders and move images
for image_name, class_label in image_to_class.items():
    image_path = images_dir / image_name
    class_folder = images_dir / class_label
    class_folder.mkdir(parents=True, exist_ok=True)

    # Check if the file exists before attempting to move it
    if image_path.exists():
        shutil.move(str(image_path), str(class_folder))
    else:
        print(f"Warning: File not found: {image_path}")

print("Dataset reorganized.")

Dataset reorganized.


In [ ]:
# Create training and testing DataLoaders as well as get a list of class names
train_dataloader, test_dataloader, class_names = setup_dataholders.create_dataloaders(train_directory=train_dir,
                                                                               test_directory=test_dir,
                                                                               data_transforms=manual_transforms, # resize, convert images to between 0 & 1 and normalize them
                                                                               batch_size=32, # set mini-batch size to 32
                                                                               workers=4) # Fixed: removed type hint from the workers argument

train_dataloader, test_dataloader, class_names

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


(<torch.utils.data.dataloader.DataLoader at 0x791fdad65350>,
 ['images'])

The next cells focus on the actual "transfer" part of taking a model from someplace else and using it for better performance. I could have done the transforms from the previous cells in a different way that is more automatic, but I wished to explore the more manual original way first.